In [ ]:
import MySQLdb
import pandas as pd
import numpy as np
from multiprocessing import Process
import time

In [ ]:
def crawl(dev_id):
    db = MySQLdb.connect(host="192.168.120.127", user="reader", passwd="trreadonly", db="city_data", port=3306, charset="utf8")
    print("crawling {0}".format(dev_id))
    cursor = db.cursor()
    cursor.execute("SELECT * FROM city_data.city_data USE INDEX (kakou_id) WHERE dev_id={0};".format(dev_id))
    a = cursor.fetchall()
    pd.DataFrame(np.array(a), columns=["id", "dev_id", "dev_name"]).to_csv("dev_id_{0}.csv".format(dev_id))
    db.close()

In [ ]:
def check_all_workers_working(list_cur_p):
    for i in range(len(list_cur_p)):
        if not list_cur_p[i].is_alive():
            return i

    return -1

In [ ]:
list_dev_id = [
    2148469, 2148468, 2148472, 2148470, 2148471, 2148740, 
    2148742, 2148741, 2148737, 2148739, 2148575, 2148572, 
    2148577, 2149892, 2149895, 2148532, 2148571, 2148574, 
    2149893, 2148573, 2149666, 2149669, 2149668, 2149667, 2149671, 2149670
             ]
n_workers = 8

process_list = []

for dev_id in list_dev_id:
    print(dev_id)
    p = Process(target=crawl, 
                args=(dev_id,))
    process_list.append(p)
    

i = 0
list_cur_p = []
for p in process_list:
    if len(list_cur_p) < n_workers:
        print(i)
        p.start()
        list_cur_p.append(p)
        i += 1
    if len(list_cur_p) < n_workers:
        continue

    idle = check_all_workers_working(list_cur_p)

    while idle == -1:
        time.sleep(1)
        idle = check_all_workers_working(
            list_cur_p)
    del list_cur_p[idle]

for p in list_cur_p:
    p.join()
    
print("finished")

In [ ]:
# import chardet
# print(dir(chardet))
# chardet.detect(a[0][1].encode())